<a href="https://colab.research.google.com/github/vincentungu00/Coding-Challenges/blob/main/Restaurant_Orders_Bundlings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the libraries
import pandas as pd
import numpy as np
import random
import datetime

pd.options.mode.copy_on_write = True

In [ ]:
# Load the processed data
fact_table = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/refs/heads/main/Star%20Schema/fact_table.csv')
dim_menu_table = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/refs/heads/main/Star%20Schema/dim_menu_table.csv')
dim_date_table = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/refs/heads/main/Star%20Schema/dim_date_table.csv')
dim_time_table = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/refs/heads/main/Star%20Schema/dim_time_table.csv')

In [ ]:
joined_df = pd.merge(fact_table, dim_menu_table, on='menu_item_id', how='inner')
joined_df = pd.merge(joined_df, dim_date_table, on='order_date_id', how='inner')
joined_df = pd.merge(joined_df, dim_time_table, on='order_time_id', how='inner')

In [ ]:
joined_df.columns

Index(['order_id', 'order_details_id', 'order_date_id', 'order_time_id',
       'menu_item_id', 'quantity', 'item_name', 'category', 'price', 'revenue',
       'order_date', 'order_date_day', 'order_date_month', 'order_date_year',
       'order_date_week', 'order_date_quarter', 'order_date_day_name',
       'order_date_weekflag', 'order_time', 'order_time_hour',
       'order_time_minute', 'order_time_second', 'order_time_am_pm',
       'order_eat_time'],
      dtype='object')

In [ ]:
joined_df

,order_id,order_details_id,order_date_id,order_time_id,menu_item_id,quantity,item_name,category,price,revenue,...,order_date_week,order_date_quarter,order_date_day_name,order_date_weekflag,order_time,order_time_hour,order_time_minute,order_time_second,order_time_am_pm,order_eat_time
0,1,1,20230101,113836,109,1,Korean Beef Bowl,Asian,17.95,8.975,...,52,1,Sunday,Weekend,11:38:36,11,38,36,AM,Brunch
1,2,1,20230101,115740,106,1,French Fries,American,7.00,3.500,...,52,1,Sunday,Weekend,11:57:40,11,57,40,AM,Brunch
2,2,2,20230101,115740,108,1,Tofu Pad Thai,Asian,14.50,7.250,...,52,1,Sunday,Weekend,11:57:40,11,57,40,AM,Brunch
3,2,3,20230101,115740,117,1,Chicken Burrito,Mexican,12.95,6.475,...,52,1,Sunday,Weekend,11:57:40,11,57,40,AM,Brunch
4,2,4,20230101,115740,124,1,Spaghetti,Italian,14.50,7.250,...,52,1,Sunday,Weekend,11:57:40,11,57,40,AM,Brunch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11785,5369,1,20230331,220504,109,1,Korean Beef Bowl,Asian,17.95,8.975,...,13,1,Friday,Weekday,22:05:04,22,5,4,PM,Evening Snack
11786,5369,2,20230331,220504,120,1,Steak Torta,Mexican,13.95,6.975,...,13,1,Friday,Weekday,22:05:04,22,5,4,PM,Evening Snack
11787,5369,3,20230331,220504,122,1,Chips & Salsa,Mexican,7.00,3.500,...,13,1,Friday,Weekday,22:05:04,22,5,4,PM,Evening Snack
11788,5369,4,20230331,220504,129,1,Mushroom Ravioli,Italian,15.50,7.750,...,13,1,Friday,Weekday,22:05:04,22,5,4,PM,Evening Snack


In [ ]:
# Preprocess data for bundlings recommendation

columns = ['order_id', 'item_name', 'category', 'price', 'revenue']

asian_orders = joined_df[joined_df['category'] == 'Asian'][columns]

non_asian_orders = joined_df[joined_df['category'] != 'Asian'][columns]

o = pd.merge(asian_orders, non_asian_orders, on='order_id', how='inner')
o['total_price'] = o['price_x'] + o['price_y']
o['total_revenue'] = o['revenue_x'] + o['revenue_y']

In [ ]:
o

,order_id,item_name_x,category_x,price_x,revenue_x,item_name_y,category_y,price_y,revenue_y,total_price,total_revenue
0,2,Tofu Pad Thai,Asian,14.50,7.250,French Fries,American,7.00,3.500,21.50,10.750
1,2,Tofu Pad Thai,Asian,14.50,7.250,Chicken Burrito,Mexican,12.95,6.475,27.45,13.725
2,2,Tofu Pad Thai,Asian,14.50,7.250,Spaghetti,Italian,14.50,7.250,29.00,14.500
3,2,Tofu Pad Thai,Asian,14.50,7.250,Mushroom Ravioli,Italian,15.50,7.750,30.00,15.000
4,6,Potstickers,Asian,9.00,4.500,Hamburger,American,12.95,6.475,21.95,10.975
...,...,...,...,...,...,...,...,...,...,...,...
5654,5368,Edamame,Asian,5.00,2.500,Steak Burrito,Mexican,14.95,7.475,19.95,9.975
5655,5368,Edamame,Asian,5.00,2.500,Cheese Lasagna,Italian,15.50,7.750,20.50,10.250
5656,5369,Korean Beef Bowl,Asian,17.95,8.975,Steak Torta,Mexican,13.95,6.975,31.90,15.950
5657,5369,Korean Beef Bowl,Asian,17.95,8.975,Chips & Salsa,Mexican,7.00,3.500,24.95,12.475


# Bundlings Recommendation

In [ ]:
american_mix = o[o['category_y'] == 'American']
american_mix.groupby(['item_name_x', 'item_name_y']).agg(distinct_order_id = ('order_id', 'nunique'), sum_price = ('total_price', 'sum'), sum_revenue = ('total_revenue', 'sum')).sort_values(by=['distinct_order_id'], ascending = False)

distinct_order_id  sum_price  sum_revenue
item_name_x      item_name_y                                             
Edamame          Hamburger                     78    1400.10      700.050
                 Cheeseburger                  75    1421.25      710.625
Korean Beef Bowl French Fries                  66    1646.70      823.350
                 Hamburger                     61    1884.90      942.450
Edamame          French Fries                  60     720.00      360.000
Korean Beef Bowl Cheeseburger                  59    1882.10      941.050
Tofu Pad Thai    French Fries                  58    1247.00      623.500
                 Hamburger                     58    1592.10      796.050
                 Cheeseburger                  58    1650.10      825.050
Orange Chicken   French Fries                  56    1316.00      658.000
Edamame          Mac & Cheese                  56     672.00      336.000
Korean Beef Bowl Mac & Cheese                  50    1247.50      623.750
California Roll  Cheeseburger                  50    1295.00      647.500
Orange Chicken   Hamburger                     45    1325.25      662.625
Salmon Roll      French Fries                  44     965.80      482.900
Tofu Pad Thai    Mac & Cheese                  43     924.50      462.250
Salmon Roll      Hamburger                     43    1199.70      599.850
Orange Chicken   Mac & Cheese                  42     987.00      493.500
Pork Ramen       Hamburger                     41    1266.90      633.450
                 Cheeseburger                  41    1307.90      653.950
California Roll  French Fries                  40     758.00      379.000
Pork Ramen       French Fries                  39     973.05      486.525
Salmon Roll      Cheeseburger                  39    1127.10      563.550
Pork Ramen       Mac & Cheese                  37     923.15      461.575
California Roll  Hamburger                     36     896.40      448.200
Orange Chicken   Cheeseburger                  36    1096.20      548.100
Tofu Pad Thai    Veggie Burger                 35     875.00      437.500
Salmon Roll      Mac & Cheese                  34     746.30      373.150
Tofu Pad Thai    Hot Dog                       31     728.50      364.250
California Roll  Mac & Cheese                  31     587.45      293.725
Korean Beef Bowl Hot Dog                       30     808.50      404.250
Edamame          Veggie Burger                 29     449.50      224.750
Potstickers      Hamburger                     29     636.55      318.275
Orange Chicken   Hot Dog                       28     714.00      357.000
Potstickers      French Fries                  25     400.00      200.000
Edamame          Hot Dog                       23     322.00      161.000
Korean Beef Bowl Veggie Burger                 22     625.90      312.950
Potstickers      Cheeseburger                  22     504.90      252.450
Orange Chicken   Veggie Burger                 20     540.00      270.000
California Roll  Hot Dog                       20     419.00      209.500
Salmon Roll      Hot Dog                       14     335.30      167.650
Potstickers      Mac & Cheese                  13     208.00      104.000
Pork Ramen       Veggie Burger                 13     369.85      184.925
California Roll  Veggie Burger                 12     269.40      134.700
Salmon Roll      Veggie Burger                 12     305.40      152.700
Potstickers      Hot Dog                       12     216.00      108.000
Pork Ramen       Hot Dog                        9     242.55      121.275
Potstickers      Veggie Burger                  5      97.50       48.750

In [ ]:
italian_mix = o[o['category_y'] == 'Italian']
italian_mix.groupby(['item_name_x', 'item_name_y']).agg(distinct_order_id = ('order_id', 'nunique'), sum_price = ('total_price', 'sum'), sum_revenue = ('total_revenue', 'sum')).sort_values(by=['distinct_order_id'], ascending = False).head(15)

,,distinct_order_id,sum_price,sum_revenue
item_name_x,item_name_y,,,
Korean Beef Bowl,Spaghetti & Meatballs,60,2154.00,1077.000
Edamame,Spaghetti & Meatballs,54,1239.30,619.650
Korean Beef Bowl,Mushroom Ravioli,50,1672.50,836.250
Edamame,Chicken Parmesan,46,1055.70,527.850
Korean Beef Bowl,Spaghetti,46,1492.70,746.350
Tofu Pad Thai,Eggplant Parmesan,45,1415.25,707.625
Orange Chicken,Spaghetti & Meatballs,45,1550.25,775.125
Korean Beef Bowl,Chicken Parmesan,45,1615.50,807.750
Tofu Pad Thai,Spaghetti & Meatballs,44,1427.80,713.900


In [ ]:
mexican_mix = o[o['category_y'] == 'Mexican']
mexican_mix.groupby(['item_name_x', 'item_name_y']).agg(distinct_order_id = ('order_id', 'nunique'), sum_price = ('total_price', 'sum'), sum_revenue = ('total_revenue', 'sum')).sort_values(by=['distinct_order_id'], ascending = False).head(15)

distinct_order_id  sum_price  sum_revenue
item_name_x      item_name_y                                               
Edamame          Chips & Salsa                   62     744.00      372.000
Tofu Pad Thai    Chicken Burrito                 59    1619.55      809.775
                 Steak Torta                     58    1650.10      825.050
Edamame          Steak Torta                     54    1023.30      511.650
                 Chicken Burrito                 52     933.40      466.700
                 Steak Burrito                   50     997.50      498.750
Korean Beef Bowl Chips & Salsa                   50    1247.50      623.750
                 Steak Torta                     49    1563.10      781.550
Orange Chicken   Steak Torta                     48    1461.60      730.800
Korean Beef Bowl Steak Burrito                   47    1546.30      773.150
Edamame          Chicken Torta                   45     762.75      381.375
Orange Chicken   Chips & Salsa                   44    1034.00      517.000
Korean Beef Bowl Chicken Torta                   44    1315.60      657.800
Tofu Pad Thai    Chips & Salsa                   44     946.00      473.000
Orange Chicken   Chicken Burrito                 43    1266.35      633.175

# Simulation

In [ ]:
menus = {'Hamburger': 6.475
         , 'Cheeseburger': 6.975
         , 'Mac & Cheese': 3.500
         , 'Spaghetti & Meatballs': 8.975
         , 'Chicken Parmesan': 8.975
         , 'Spaghetti': 7.250
         , 'Steak Torta': 6.975
         , 'Chicken Burrito': 6.475
         , 'Steak Burrito': 7.475
}

In [ ]:
joined_df_v2 = joined_df.copy()
conversion_rate = 0.5  # Assume 50% of the customers who are eligible for the simulation interested in the bundlings.


# Orders with ex-Edamame -- Assume customer who initially purchased heavy meal become interested in adding Edamame
selected_menus = joined_df_v2[joined_df_v2['item_name'].isin(list(menus.keys()))]['order_id'].unique()

order_counts = joined_df_v2.groupby('order_id')['item_name'].count()
single_item_orders = order_counts[order_counts == 1].index

intersect_orders = np.intersect1d(selected_menus, single_item_orders)

num_samples = int(conversion_rate * len(intersect_orders))
np.random.seed(42)
sampled_order_ids = np.random.choice(intersect_orders, size=num_samples, replace=False)

for order_id in sampled_order_ids:
  for item_name in menus:
    # Find rows matching the current order_id and item_name
    rows_to_update = joined_df_v2[(joined_df_v2['order_id'] == order_id) & (joined_df_v2['item_name'] == item_name)]

    # Check if any rows were found
    if not rows_to_update.empty:
      # Update the price for the found rows
      joined_df_v2.loc[rows_to_update.index, 'revenue'] += 1.75 # Add Edamame discounted price



# Orders with Edamame only -- Assume customer who initially purchased Edamame only become interested in one of the bundlings
selected_menus = joined_df_v2[joined_df_v2['item_name'].isin(['Edamame'])]['order_id'].unique()

order_counts = joined_df_v2.groupby('order_id')['item_name'].count()
single_item_orders = order_counts[order_counts == 1].index

intersect_orders = np.intersect1d(selected_menus, single_item_orders)

num_samples = int(conversion_rate * len(intersect_orders))
np.random.seed(42)
sampled_order_ids = np.random.choice(intersect_orders, size=num_samples, replace=False)

np.random.seed(42)
random_menu_items = np.random.choice(list(menus.keys()), size=len(sampled_order_ids), replace=True)

for i, order_id in enumerate(sampled_order_ids):
  rows_to_update = joined_df_v2[joined_df_v2['order_id'] == order_id]

  joined_df_v2.loc[rows_to_update.index, 'revenue'] += menus[random_menu_items[i]]
  joined_df_v2.loc[rows_to_update.index, 'revenue'] -= 0.75 # Substract Edamame discounted price

# Calculate total revenue per date
revenue_per_date = joined_df_v2.groupby('order_date').agg({'revenue': 'sum'})

# Print the result
revenue_per_date

,revenue
order_date,
2023-01-01,1024.150
2023-01-02,994.350
2023-01-03,982.875
2023-01-04,680.700
2023-01-05,794.950
...,...
2023-03-27,1060.475
2023-03-28,870.425
2023-03-29,855.825


In [ ]:
# Calculate total revenue per date
revenue_per_date = joined_df.groupby('order_date').agg({'revenue': 'sum'})

revenue_per_date

,revenue
order_date,
2023-01-01,1018.900
2023-01-02,979.125
2023-01-03,971.400
2023-01-04,665.475
2023-01-05,787.950
...,...
2023-03-27,1055.225
2023-03-28,863.425
2023-03-29,843.350
